In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pymongo import MongoClient

In [ ]:
client = MongoClient('127.0.0.1',
                     username="root",
                     password="rootpassword")

collection = client['performance'].get_collection("perf")

data = pd.DataFrame(list(collection.find()))
data.head()

In [ ]:
parse_dates = ['submit_time', 'start_time', 'end_time']
for d in parse_dates:
    data[d] = pd.to_datetime(data[d])

convert_dtype = {'_id': "string",
                 'compute_platform': "string",
                 'requested_cpu': 'int8',
                 'requested_mem': 'int8',
                 'requested_disk': 'int8',
                 'input_file_size': 'int64',
                 'input_file_compression': 'bool',
                 'input_file_time': 'float64',
                 'pipeline': 'string',
                 'output_file_size': 'int64',
                 'output_file_compression': 'bool',
                 'output_file_time': 'float64'}
    
for d, t in convert_dtype.items():
    data[d] = data[d].astype(t)

In [ ]:
dict(data.dtypes)

In [ ]:
data.info(verbose=True, memory_usage='deep')

In [ ]:
data.head()

In [ ]:
platform = data.compute_platform.unique()
plt.hist([data.loc[data.compute_platform == x, 'requested_cpu'] for x in platform], label=platform)
plt.legend()

In [ ]:
data['full_runtime'] = data.end_time - data.submit_time
data['full_runtime'] = data['full_runtime'].apply(lambda x: x.value)

In [ ]:
plt.hist(data.full_runtime, bins=30)
plt.show()

In [ ]:
platform = data.compute_platform.unique()
plt.hist([data.loc[data.compute_platform == x, 'full_runtime'] for x in platform], label=platform, bins=20)
plt.legend()